<a href="https://colab.research.google.com/github/jselvarathinam/cse5334/blob/main/CSE5334_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Packages and Data

In [1]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Importing Necessary packages
import os
import tensorflow as tf
import random
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import zipfile
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, shutil, math, cv2, json, random
from tqdm import tqdm
from keras.layers import *
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [4]:
"""
The following four cells make a kaggle directory in this workspace and download the caltech101 dataset into it.
I learned how to do this from the following youtube tutorial: https://www.youtube.com/watch?v=yEXkEUqK52Q. I think 
this code is fairly common but I wanted to include a reference just in case.
"""
! mkdir ~/.kaggle

In [5]:
! cp /content/drive/MyDrive/ColabNotebooks/kaggle.json ~/.kaggle/kaggle.json

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [13]:
! kaggle datasets download -d whenamancodes/wild-animals-images #Need to download only specific folders
! mkdir wild-animals-images
! mv wild-animals-images.zip wild-animals-images
%cd wild-animals-images

wild-animals-images.zip: Skipping, found more recently modified local copy (use --force to force download)
/content/wild-animals-images


In [14]:
! unzip wild-animals-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: cheetah-resize-224/resize-224/00000236_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000239_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000240_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000242_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000244_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000245_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000246_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000247_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000248_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000250_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000251_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000252_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000254_224resized.png  
  inflating: cheetah-resize-224/resize-224/00000255_

In [15]:
! rm -r cheetah-resize-224
! rm -r cheetah-resize-512
! rm -r fox-resize-224
! rm -r fox-resize-512
! rm -r hyena-resize-224
! rm -r hyena-resize-512
! rm -r lion-resize-224
! rm -r lion-resize-512
! rm -r tiger-resize-224
! rm -r tiger-resize-512
! rm -r wolf-resize-224
! rm -r wolf-resize-512

In [16]:
%cd /content/wild-animals-images/wild-animals-images
! mkdir cheetah
! mkdir fox
! mkdir hyena
! mkdir lion
! mkdir tiger
! mkdir wolf

[Errno 2] No such file or directory: '/content/wild-animals-images/wild-animals-images'
/content/wild-animals-images


In [19]:
! mv /content/wild-animals-images/cheetah-resize-300/resize-300/* /content/wild-animals-images/cheetah
! mv /content/wild-animals-images/fox-resize-300/fox-resize-300/* /content/wild-animals-images/fox
! mv /content/wild-animals-images/hyena-resize-300/resize-300/* /content/wild-animals-images/hyena
! mv /content/wild-animals-images/lion-resize-300/lion-resize-300/* /content/wild-animals-images/lion
! mv /content/wild-animals-images/tiger-resize-300/tiger-resize-300/* /content/wild-animals-images/tiger
! mv /content/wild-animals-images/wolf-resize-300/wolf-resize-300/* /content/wild-animals-images/wolf

! rm -r /content/wild-animals-images/cheetah-resize-300
! rm -r /content/wild-animals-images/fox-resize-300
! rm -r /content/wild-animals-images/hyena-resize-300
! rm -r /content/wild-animals-images/lion-resize-300
! rm -r /content/wild-animals-images/tiger-resize-300
! rm -r /content/wild-animals-images/wolf-resize-300
! rm -r /content/wild-animals-images/wild-animals-images.zip

# Creating Training, Testing, Validation Data on 80/10/10 split


In [21]:
'''
This code in this cell is from https://www.kaggle.com/code/jiaowoguanren/airplanes-motorbikes-schooners-tf-efficientnet 
I used it to create a train/test/validate split for my data. (It also helped me in the next cell to understand I needed to use the 
".flow_from_dataframe" when generating batches of images and labels for training, testing and validating the neural net). 
'''
sdir = '/content/wild-animals-images'

#Defining function that creates training, testing, and validation dataframes
def make_dataframes(sdir):
    bad_images = []
    good_ext = ['jpg', 'jpeg', 'png', 'tiff']
    filepaths = []
    labels = []
    classes = sorted(os.listdir(sdir))
    for klass in classes:
        classpath = os.path.join(sdir, klass)
        flist = sorted(os.listdir(classpath))
        desc = f'{klass:23s}'
        for f in tqdm(flist, ncols=110, desc=desc, unit='file', colour='blue'):
            fpath = os.path.join(classpath, f)
            fl = f.lower()
            index = fl.rfind('.')
            ext = fl[index + 1:]
            if ext in good_ext:
                try:
                    img = cv2.imread(fpath)
                    shape = img.shape
                    filepaths.append(fpath)
                    labels.append(klass)
                except:
                    bad_images.append(fpath)
                    print('defective image file: ', fpath)
            else:
                bad_images.append(fpath)
    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')
    df = pd.concat([Fseries, Lseries], axis=1)

    train_df, dummy_df = train_test_split(df, train_size=.8, shuffle=True, random_state=123, stratify=df['labels'])
    valid_df, test_df = train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123,
                                         stratify=dummy_df['labels'])
    classes = sorted(train_df['labels'].unique())
    class_count = len(classes)
    sample_df = train_df.sample(n=50, replace=False)

    ht = 0
    wt = 0
    count = 0
    
    for i in range(len(sample_df)):
        fpath = sample_df['filepaths'].iloc[i]
        try:
            img = cv2.imread(fpath)
            h = img.shape[0]
            w = img.shape[1]
            wt += w
            ht += h
            count += 1
        except:
            pass
    have = int(ht / count)
    wave = int(wt / count)
    aspect_ratio = have / wave
    print('number of classes in processed dataset= ', class_count)
    counts = list(train_df['labels'].value_counts())
    print('the maximum files in any class in train_df is ', max(counts),
          '  the minimum files in any class in train_df is ', min(counts))
    print('train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))
    print('average image height= ', have, '  average image width= ', wave, ' aspect ratio h/w= ', aspect_ratio)
    return train_df, test_df, valid_df, classes, class_count

#Create training, testing, and validation dataframes
train_df, test_df, valid_df, classes, class_count = make_dataframes(sdir)

cheetah                :  62%|██████████████████████████▋                | 213/343 [00:00<00:00, 240.33file/s]

defective image file:  /content/wild-animals-images/cheetah/00000244_300resized.png


hyena                  :  65%|███████████████████████████▊               | 198/306 [00:00<00:00, 240.37file/s]

defective image file:  /content/wild-animals-images/hyena/00000224_300resized.png


wolf                   : 100%|███████████████████████████████████████████| 263/263 [00:01<00:00, 242.55file/s]


number of classes in processed dataset=  6
the maximum files in any class in train_df is  274   the minimum files in any class in train_df is  200
train_df length:  1378   test_df length:  173   valid_df length:  172
average image height=  300   average image width=  300  aspect ratio h/w=  1.0


In [22]:
"""
The code in this cell and the next 2 cells is loosely based on the code in the following tutorial: 
https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%208%20-%20Lesson%202%20-%20Notebook%20(RockPaperScissors).ipynb#scrollTo=ZABJp7T3VLCU
I say loosely based because while I learned the essence of how to set up, compile and fit the model from this tutorial it would not have worked for the problem I am trying to solve,
I had to learn what all the code, parameters and hyperparamters meant from documentation and tutorials (that I've linked in my blog post) and then write code that would actually work for 
my particular problem. Therefore these next few cells are where most of my unique contributions are.
"""
#Rescaling images
train_datagen = ImageDataGenerator(rescale= 1.0/255)
test_datagen = ImageDataGenerator(rescale= 1.0/255)
valid_datagen = ImageDataGenerator(rescale= 1.0/255) 

#Generating batches of images for training, validation, and testing along with their labels to be fed into the neural net
#Read documentation on .flow_from_dataframe() from tensorflow website: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_dataframe
train_data = train_datagen.flow_from_dataframe(train_df, x_col = "filepaths", y_col = "labels", batch_size =50, class_mode= "categorical", target_size = (224,224))
test_data = test_datagen.flow_from_dataframe(test_df, x_col = "filepaths", y_col = "labels", batch_size =50, class_mode= "categorical", target_size = (224,224))
valid_data = valid_datagen.flow_from_dataframe(valid_df, x_col = "filepaths", y_col = "labels", batch_size =50, class_mode= "categorical", target_size = (224,224))

Found 1378 validated image filenames belonging to 6 classes.
Found 173 validated image filenames belonging to 6 classes.
Found 172 validated image filenames belonging to 6 classes.


In [23]:
#Building the model
model = tf.keras.models.Sequential([
    
    #First convolution Layer
    tf.keras.layers.Conv2D(16, (3,3), activation= "relu", input_shape= (224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    #Second convolution layer
    tf.keras.layers.Conv2D(32, (3,3), activation= "relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    #Flattening to feed into DNN
    tf.keras.layers.Flatten(),

    #Hidden Layer with 512 neurons
    tf.keras.layers.Dense(512, activation= 'relu'),

    #Output layer
    tf.keras.layers.Dense(3, activation= 'softmax')

    ])

model.compile(loss= "mean_squared_error", optimizer= RMSprop(lr=0.001), metrics = ["acc"])


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [25]:
#Fitting the model
history = model.fit(train_data, 
                    epochs = 20, 
                    steps_per_epoch = 5, 
                    validation_data = valid_data, 
                    validation_steps = 5)

Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
'''
The basics of the code for creating graphs of the training and validation accuracy per epoch came from the following tutorial provided by Google:
 https://colab.research.google.com/github/lmoroney/mlday-tokyo/blob/master/Lab6-Cats-v-Dogs.ipynb#scrollTo=dn-6c02VmqiN
 I added to the code to make the graphs more descriptive.
'''

# Retrieve a list of list results on training and test datasets for each training epoch
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

# Get number of epochs
epochs= range(len(acc)) 

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, 'r', label = "Training Accuracy")
plt.plot(epochs, val_acc, 'b', label = "Validation Accuracy")
plt.title('Training and validation accuracy per epoch')
plt.legend(loc= 2)
plt.figure
plt.show()

print(" ")   #Inserting a space between plots

# Plot training and validation loss per epoch
plt.plot(epochs, loss, 'r', label = "Training Loss")
plt.plot(epochs, val_loss, 'b', label =  "Validation Loss")
plt.title('Training and validation loss per epoch')
plt.legend(loc= 0)
plt.figure()
plt.show()

# 